In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
from datasets import load_dataset

dataset = load_dataset("poem_sentiment")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/892 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/105 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/104 [00:00<?, ? examples/s]

In [ ]:
# Assuming you want to convert the 'train' split to a pandas DataFrame
# First, check what splits are available if you're not sure
print(dataset.keys())


dict_keys(['train', 'validation', 'test'])


In [ ]:

# Convert the desired split to a pandas DataFrame
# Replace 'train' with the appropriate split if it's different
train_df = dataset['train'].to_pandas()
val_df = dataset['validation'].to_pandas()
test_df = dataset['test'].to_pandas()

In [ ]:
train_df

,id,verse_text,label
0,0,with pale blue berries. in these peaceful shad...,1
1,1,"it flows so long as falls the rain,",2
2,2,"and that is why, the lonesome day,",0
3,3,"when i peruse the conquered fame of heroes, an...",3
4,4,of inward strife for truth and liberty.,3
...,...,...,...
887,887,to his ears there came a murmur of far seas be...,2
888,888,"the one good man in the world who knows me, --",1
889,889,faint voices lifted shrill with pain,0
890,890,"an', fust you knowed on, back come charles the...",2


In [ ]:
train_df.to_csv('gpt_train.csv')
val_df.to_csv('gpt_val.csv')
test_df.to_csv('gpt_test.csv')

In [10]:
from datasets import load_dataset

dataset_am_train = load_dataset("FanFan/sentiment-amazon-clean")

In [11]:
# Assuming you want to convert the 'train' split to a pandas DataFrame
# First, check what splits are available if you're not sure
print(dataset_am_train.keys())


dict_keys(['train'])


In [73]:
am_train = dataset_am_train['train'].to_pandas().copy()

In [74]:
am_train

,inputs,prediction,prediction_agent,annotation,annotation_agent,multi_label,explanation,id,metadata,status,event_timestamp,metrics
0,{'text': 'five stars - son loves always'},"[{'label': 'NEGATIVE', 'score': 0.000569589086...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'review_id': 'R3GAU6LZQ6STAA'},Default,None,None
1,{'text': 'excellent - excellent recorder sensi...,"[{'label': 'NEGATIVE', 'score': 0.696283638477...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'review_id': 'R3DCCYH3TBAT0X'},Default,None,None
2,{'text': 'quick shipping - product shipped qui...,"[{'label': 'NEGATIVE', 'score': 0.004670690745...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'review_id': 'R1RN0EUC1G8IDB'},Default,None,None
3,{'text': 'amazing deal works - little worried ...,"[{'label': 'NEGATIVE', 'score': 0.469087600708...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'review_id': 'R1NU1RRASP1AHO'},Default,None,None
4,{'text': 'not worth - bought thinking getting ...,"[{'label': 'NEGATIVE', 'score': 0.995140314102...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'review_id': 'R3MV12QAJVPEU7'},Default,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
104970,{'text': 'three stars - not expected'},"[{'label': 'NEGATIVE', 'score': 0.008588143624...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'review_id': 'R24BQNW31X397F'},Default,None,None
104971,{'text': 'terrible product - bought two years ...,"[{'label': 'NEGATIVE', 'score': 0.998383522033...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'review_id': 'R2Y5AQPBFVINCX'},Default,None,None
104972,{'text': 'great itehem great price - reading r...,"[{'label': 'NEGATIVE', 'score': 0.736952543258...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'review_id': 'R183PXD8391IWG'},Default,None,None
104973,{'text': 'sounds great - gift son loves impres...,"[{'label': 'NEGATIVE', 'score': 0.000822546542...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'review_id': 'R1J47G1MDX222P'},Default,None,None


In [75]:
selected_columns = ['inputs', 'prediction', 'metadata']
am_train = am_train[selected_columns]

In [76]:
am_train_df=am_train

In [69]:
# output_path = '/content/am_train_df.parquet'
# am_train_df.to_parquet(output_path, index=False)

In [77]:
# Assuming 'inputs' column contains dictionaries with a 'text' key
am_train_df['inputs'] = am_train_df['inputs'].apply(lambda x: x.get('text', None))

# Task 1: Extracting values from the 'inputs' column
# am_train_df['inputs'] = am_train_df['inputs'].apply(lambda x: x['text']).copy()  # Use .copy()

# Display the resulting DataFrame
print(am_train_df[['inputs']])

def extract_negative_score(prediction_list):
    return next((pred['score'] for pred in prediction_list if pred['label'] == 'NEGATIVE'), None)

def extract_positive_score(prediction_list):
    return next((pred['score'] for pred in prediction_list if pred['label'] == 'POSITIVE'), None)

# 'prediction' 열을 업데이트
# am_train_df['neg'] = am_train_df['prediction'].apply(extract_negative_score)
# am_train_df['pos'] = am_train_df['prediction'].apply(extract_positive_score)


def extract_label(prediction_list):
    score = next((pred['score'] for pred in prediction_list if pred['label'] == 'POSITIVE'), None)
    if score is not None:
        label = 1 if score >= 0.5 else 0
        return label
    else:
        return None

def classify_positive_score(prediction_list):
    score = next((pred['score'] for pred in prediction_list if pred['label'] == 'POSITIVE'), None)
    if score is not None:
        if score < 0.4:
            return 0
        elif 0.4 <= score <= 0.6:
            return 2
        else:  # score > 0.6
            return 1
    else:
        return None


# 'prediction' 열을 업데이트
am_train_df['generated'] = am_train_df['prediction'].apply(extract_label)

print(am_train_df)

# Extracting 'review_id' from 'metadata' and creating a new 'id' column
am_train_df['id'] = am_train_df['metadata'].apply(lambda x: x.get('review_id', None))

# Now, if you want to drop the 'prediction' and 'metadata' columns
am_train_df = am_train_df.drop(['prediction', 'metadata'], axis=1)

print(am_train_df)

                                                   inputs
0                           five stars - son loves always
1       excellent - excellent recorder sensitive mic c...
2       quick shipping - product shipped quickly despi...
3       amazing deal works - little worried buying wit...
4       not worth - bought thinking getting good deal ...
...                                                   ...
104970                         three stars - not expected
104971  terrible product - bought two years ago one gi...
104972  great itehem great price - reading reviews sit...
104973  sounds great - gift son loves impressed good s...
104974  great cases - fits great even screen protector...

[104975 rows x 1 columns]
                                                   inputs  \
0                           five stars - son loves always   
1       excellent - excellent recorder sensitive mic c...   
2       quick shipping - product shipped quickly despi...   
3       amazing deal works - litt

<ipython-input-77-833f4b20dcd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  am_train_df['inputs'] = am_train_df['inputs'].apply(lambda x: x.get('text', None))
<ipython-input-77-833f4b20dcd7>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  am_train_df['generated'] = am_train_df['prediction'].apply(extract_label)
<ipython-input-77-833f4b20dcd7>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [78]:
am_train_df

,inputs,generated,id
0,five stars - son loves always,1,R3GAU6LZQ6STAA
1,excellent - excellent recorder sensitive mic c...,0,R3DCCYH3TBAT0X
2,quick shipping - product shipped quickly despi...,1,R1RN0EUC1G8IDB
3,amazing deal works - little worried buying wit...,1,R1NU1RRASP1AHO
4,not worth - bought thinking getting good deal ...,0,R3MV12QAJVPEU7
...,...,...,...
104970,three stars - not expected,1,R24BQNW31X397F
104971,terrible product - bought two years ago one gi...,0,R2Y5AQPBFVINCX
104972,great itehem great price - reading reviews sit...,0,R183PXD8391IWG
104973,sounds great - gift son loves impressed good s...,1,R1J47G1MDX222P


In [59]:
# prompt: 내 구글드라이브에 접근하고싶어.

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [79]:
import pandas as pd
import os

# 파일 경로 설정 (예시 경로입니다. 실제 경로로 변경해야 합니다.)
file_paths = ['/content/drive/MyDrive/PBL중간과제/gpt번역/1~4999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/5000~9999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/10000~14999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/15000~19999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/20000~24999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/25000~29999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/30000~34999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/35000~39999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/40000~44999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/45000~49999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/50000~54999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/55000~59999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/60000~64999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/65000~69999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/70000~74999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/75000~79999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/80000~84999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/85000~89999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/90000~94999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/95000~99999 ko.xlsx',
              '/content/drive/MyDrive/PBL중간과제/gpt번역/100000~104976 ko.xlsx'
              ]


# 빈 DataFrame 생성
train_df = pd.DataFrame(columns=['text'])

# 각 파일을 순회하면서 DataFrame에 추가
for file_path in file_paths:
    # Excel 파일 읽기, 첫 번째 열을 'text'로 가정합니다.
    temp_df = pd.read_excel(file_path, header=None, names=['text'])

    # temp_df를 train_df에 추가
    train_df = pd.concat([train_df, temp_df], ignore_index=True)

# 결과 확인
print(train_df.head())


                                                text
0                               별 다섯개 - 아들은 항상 사랑합니다
1   훌륭함 - 뛰어난 녹음기 민감한 마이크 깨끗한 음성 쉬운 전송 PC 각각의 파일 ...
2   빠른 배송 - 5일이 걸릴 수 있다는 말에도 불구하고 제품이 빨리 배송되었습니다....
3   놀라운 거래가 작동합니다 - 피드백 없이 구매하면 걱정할 필요가 거의 없습니다. ...
4  가치 없음 - 좋은 거래를 하면 이베이 달러를 심각하게 얻을 수 있다고 생각하고 구...


In [80]:
train_df

,text
0,별 다섯개 - 아들은 항상 사랑합니다
1,훌륭함 - 뛰어난 녹음기 민감한 마이크 깨끗한 음성 쉬운 전송 PC 각각의 파일 ...
2,빠른 배송 - 5일이 걸릴 수 있다는 말에도 불구하고 제품이 빨리 배송되었습니다....
3,놀라운 거래가 작동합니다 - 피드백 없이 구매하면 걱정할 필요가 거의 없습니다. ...
4,가치 없음 - 좋은 거래를 하면 이베이 달러를 심각하게 얻을 수 있다고 생각하고 구...
...,...
104970,별 3개 - 예상되지 않음
104971,형편없는 제품 - 2년 전에 구입한 선물 하나 교환 딸은 전혀 좋아하지 않았습니다...
104972,좋은 아이템 좋은 가격 - 독서 리뷰 사이트 정착 차량 행복 첫 번째 쉬운 사용 ...
104973,좋은 것 같아요 - 선물 아들이 감동받은 것을 좋아합니다 좋은 소리 강력한 추천 신...


In [81]:
am_train_df

,inputs,generated,id
0,five stars - son loves always,1,R3GAU6LZQ6STAA
1,excellent - excellent recorder sensitive mic c...,0,R3DCCYH3TBAT0X
2,quick shipping - product shipped quickly despi...,1,R1RN0EUC1G8IDB
3,amazing deal works - little worried buying wit...,1,R1NU1RRASP1AHO
4,not worth - bought thinking getting good deal ...,0,R3MV12QAJVPEU7
...,...,...,...
104970,three stars - not expected,1,R24BQNW31X397F
104971,terrible product - bought two years ago one gi...,0,R2Y5AQPBFVINCX
104972,great itehem great price - reading reviews sit...,0,R183PXD8391IWG
104973,sounds great - gift son loves impressed good s...,1,R1J47G1MDX222P


In [82]:
# axis=1은 열을 기준으로 데이터 프레임을 합치라는 것을 의미합니다.
combined_df = pd.concat([am_train_df, train_df['text']], axis=1)

In [83]:
combined_df

,inputs,generated,id,text
0,five stars - son loves always,1,R3GAU6LZQ6STAA,별 다섯개 - 아들은 항상 사랑합니다
1,excellent - excellent recorder sensitive mic c...,0,R3DCCYH3TBAT0X,훌륭함 - 뛰어난 녹음기 민감한 마이크 깨끗한 음성 쉬운 전송 PC 각각의 파일 ...
2,quick shipping - product shipped quickly despi...,1,R1RN0EUC1G8IDB,빠른 배송 - 5일이 걸릴 수 있다는 말에도 불구하고 제품이 빨리 배송되었습니다....
3,amazing deal works - little worried buying wit...,1,R1NU1RRASP1AHO,놀라운 거래가 작동합니다 - 피드백 없이 구매하면 걱정할 필요가 거의 없습니다. ...
4,not worth - bought thinking getting good deal ...,0,R3MV12QAJVPEU7,가치 없음 - 좋은 거래를 하면 이베이 달러를 심각하게 얻을 수 있다고 생각하고 구...
...,...,...,...,...
104970,three stars - not expected,1,R24BQNW31X397F,별 3개 - 예상되지 않음
104971,terrible product - bought two years ago one gi...,0,R2Y5AQPBFVINCX,형편없는 제품 - 2년 전에 구입한 선물 하나 교환 딸은 전혀 좋아하지 않았습니다...
104972,great itehem great price - reading reviews sit...,0,R183PXD8391IWG,좋은 아이템 좋은 가격 - 독서 리뷰 사이트 정착 차량 행복 첫 번째 쉬운 사용 ...
104973,sounds great - gift son loves impressed good s...,1,R1J47G1MDX222P,좋은 것 같아요 - 선물 아들이 감동받은 것을 좋아합니다 좋은 소리 강력한 추천 신...


In [84]:
combined_df.to_csv('combined_train.csv')

dict_keys(['train'])


In [85]:
from datasets import load_dataset

dataset_am_test = load_dataset("FanFan/sentiment-amazon-test")

Generating train split:   0%|          | 0/10 [00:00<?, ? examples/s]

In [86]:
print(dataset_am_test.keys())

dict_keys(['train'])


In [87]:
am_test_df = dataset_am_test['train'].to_pandas()

In [88]:
am_test_df

,inputs,prediction,prediction_agent,annotation,annotation_agent,multi_label,explanation,id,metadata,status,event_timestamp,metrics
0,{'text': 'This is my first Ipod Touch and I'm ...,"[{'label': 'NEGATIVE', 'score': 0.000359892525...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 5},Default,None,None
1,{'text': 'I really like the bundle idea as it ...,"[{'label': 'NEGATIVE', 'score': 0.001953014172...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 4},Default,None,None
2,{'text': 'Meh. They die easily and have to be ...,"[{'label': 'NEGATIVE', 'score': 0.975844502449...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 2},Default,None,None
3,{'text': 'Even though I have put scores of scr...,"[{'label': 'NEGATIVE', 'score': 0.960179686546...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 1},Default,None,None
4,{'text': 'Once the ipod was opened the battery...,"[{'label': 'NEGATIVE', 'score': 0.995163679122...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 4},Default,None,None
5,"{'text': 'At the MIKLOR website, http://www.mi...","[{'label': 'NEGATIVE', 'score': 0.988010466098...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 5},Default,None,None
6,"{'text': 'The cover, gel skin and screen prote...","[{'label': 'NEGATIVE', 'score': 0.000122438053...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 5},Default,None,None
7,{'text': 'Everything came in package as descri...,"[{'label': 'NEGATIVE', 'score': 0.996111571788...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 5},Default,None,None
8,{'text': 'I cannot say enough good things abou...,"[{'label': 'NEGATIVE', 'score': 0.994832515716...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 5},Default,None,None
9,{'text': 'This watch is exactly what I was exp...,"[{'label': 'NEGATIVE', 'score': 0.000626808032...",distilbert-base-uncased-finetuned-sst-2-english,None,None,False,None,None,{'category': 5},Default,None,None


In [89]:
selected_columns = ['inputs', 'prediction', 'metadata']
am_test_df = am_test_df[selected_columns]

In [ ]:
# output_path = '/content/am_test_df.parquet'
# am_test_df.to_parquet(output_path, index=False)

In [ ]:
am_test_df

,inputs,prediction,metadata
0,{'text': 'This is my first Ipod Touch and I'm ...,"[{'label': 'NEGATIVE', 'score': 0.000359892525...",{'category': 5}
1,{'text': 'I really like the bundle idea as it ...,"[{'label': 'NEGATIVE', 'score': 0.001953014172...",{'category': 4}
2,{'text': 'Meh. They die easily and have to be ...,"[{'label': 'NEGATIVE', 'score': 0.975844502449...",{'category': 2}
3,{'text': 'Even though I have put scores of scr...,"[{'label': 'NEGATIVE', 'score': 0.960179686546...",{'category': 1}
4,{'text': 'Once the ipod was opened the battery...,"[{'label': 'NEGATIVE', 'score': 0.995163679122...",{'category': 4}
5,"{'text': 'At the MIKLOR website, http://www.mi...","[{'label': 'NEGATIVE', 'score': 0.988010466098...",{'category': 5}
6,"{'text': 'The cover, gel skin and screen prote...","[{'label': 'NEGATIVE', 'score': 0.000122438053...",{'category': 5}
7,{'text': 'Everything came in package as descri...,"[{'label': 'NEGATIVE', 'score': 0.996111571788...",{'category': 5}
8,{'text': 'I cannot say enough good things abou...,"[{'label': 'NEGATIVE', 'score': 0.994832515716...",{'category': 5}
9,{'text': 'This watch is exactly what I was exp...,"[{'label': 'NEGATIVE', 'score': 0.000626808032...",{'category': 5}


In [90]:
# Task 1: Extracting values from the 'inputs' column
am_test_df['text'] = am_test_df['inputs'].apply(lambda x: x['text']).copy()  # Use .copy()

# Display the resulting DataFrame
print(am_test_df[['text']])

# Task 2: Extracting NEGATIVE or POSITIVE based on the prediction probabilities
am_test_df['generated'] = am_test_df['prediction'].apply(extract_positive_score)  # Use .copy()

# Display the resulting DataFrame
print(am_test_df[['text', 'generated']])


                                                text
0  This is my first Ipod Touch and I'm loving it....
1  I really like the bundle idea as it make shopp...
2  Meh. They die easily and have to be recharged ...
3  Even though I have put scores of screen protec...
4  Once the ipod was opened the battery was repla...
5  At the MIKLOR website, http://www.miklor.com<b...
6  The cover, gel skin and screen protector is fa...
7  Everything came in package as described. No in...
8  I cannot say enough good things about my Sony ...
9  This watch is exactly what I was expecting. It...
                                                text  generated
0  This is my first Ipod Touch and I'm loving it....   0.999640
1  I really like the bundle idea as it make shopp...   0.998047
2  Meh. They die easily and have to be recharged ...   0.024156
3  Even though I have put scores of screen protec...   0.039820
4  Once the ipod was opened the battery was repla...   0.004836
5  At the MIKLOR website, http://

<ipython-input-90-fc3e74dfc6d2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  am_test_df['text'] = am_test_df['inputs'].apply(lambda x: x['text']).copy()  # Use .copy()
<ipython-input-90-fc3e74dfc6d2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  am_test_df['generated'] = am_test_df['prediction'].apply(extract_positive_score)  # Use .copy()


In [ ]:
# am_test_df.drop('prediction','metadata',axis=0,inplace=True)

<ipython-input-65-368fb7e2e416>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  am_test_df.drop('prediction','metadata',axis=0,inplace=True)


TypeError: DataFrame.drop() got multiple values for argument 'axis'

In [93]:
selected_columns = ['text', 'generated']
am_test_df = am_test_df[selected_columns]

In [94]:
am_test_df

,text,generated
0,This is my first Ipod Touch and I'm loving it....,0.999640
1,I really like the bundle idea as it make shopp...,0.998047
2,Meh. They die easily and have to be recharged ...,0.024156
3,Even though I have put scores of screen protec...,0.039820
4,Once the ipod was opened the battery was repla...,0.004836
5,"At the MIKLOR website, http://www.miklor.com<b...",0.011989
6,"The cover, gel skin and screen protector is fa...",0.999878
7,Everything came in package as described. No in...,0.003889
8,I cannot say enough good things about my Sony ...,0.005168
9,This watch is exactly what I was expecting. It...,0.999373


In [95]:
am_test_df.to_csv('ama_test.csv')

In [ ]:
# prompt:     train_df, valid_df, train_labels, valid_labels = train_test_split(     df, test_size=0.01, random_state=42) 오류 수정해줘

train_df, valid_df, train_labels, valid_labels = train_test_split(combined_df, combined_df['generated'], test_size=0.01, random_state=42)


In [ ]:
pip install gradio

In [ ]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch(debug=True, share=True)

In [ ]:
def sepia(input_img):
    sepia_filter = np.array([
        [0.272, 0.534, 0.131],
        [0.349, 0.686, 0.168],
        [0.393, 0.769, 0.189]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

iface = gr.Interface(fn=sepia, inputs="image", outputs="image")
iface.launch(debug=True, share=True)

In [ ]:

demo = gr.Interface(fn=predict, inputs="text", outputs="text")
demo.launch()